In [22]:
import csv,sys
import math,random
import subprocess
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from sklearn.decomposition import KernelPCA

In [23]:
class Point:
    #An point in n dimensional space
    def __init__(self, coords,label):
    #coords - A list of values, one per dimension
        self.coords = coords
        self.label = label
        self.n = len(coords)

    def __repr__(self):
        return str(self.coords)

In [24]:
class Cluster:
    #A set of points and their centroid

    def __init__(self, points):
    #points - A list of point objects

        if len(points) == 0: 
            raise Exception("ILLEGAL: empty cluster")
        # The points that belong to this cluster
        self.points = points

        # The dimensionality of the points in this cluster
        self.n = points[0].n
        self.length = 1
    
        # Assert that all points are of the same dimensionality
        for p in points:
            if p.n != self.n: raise Exception("ILLEGAL: wrong dimensions")
    
        # Set up the initial centroid (this is usually based off one point)
        self.centroid = self.calculateCentroid()

    def __repr__(self):
    #String representation of this object
        return str(self.points)

    def update(self, points):
    #Returns the distance between the previous centroid and the new after
    #recalculating and storing the new centroid.
        old_centroid = self.centroid
        self.points = points
        self.centroid = self.calculateCentroid()
        shift = getDistance(old_centroid, self.centroid) 
        return shift

    def calculateCentroid(self):
    #Finds a virtual center point for a group of n-dimensional points
        numPoints = len(self.points)
        # Get a list of all coordinates in this cluster
        coords = [p.coords for p in self.points]
        # Reformat that so all x's are together, all y'z etc.
        unzipped = zip(*coords)
        # Calculate the mean for each dimension
        centroid_coords = [math.fsum(dList)/numPoints for dList in unzipped]

        return Point(centroid_coords,0)


In [25]:
def getDistance(a, b):

#Euclidean distance between two n-dimensional points.
#Note: This can be very slow and does not scale well
    #if a.n != b.n:
    #    raise Exception("ILLEGAL: non comparable points")

    ret = reduce(lambda x,y: x + pow((a.coords[y]-b.coords[y]), 2),range(a.n),0.0)
    return math.sqrt(ret)

In [26]:
def kmeans(points, k, cutoff):

    # Pick out k random points to use as our initial centroids
    initial = random.sample(points, k)

    # Create k clusters using those centroids
    clusters = [Cluster([p]) for p in initial]

    # Loop through the dataset until the clusters stabilize
    loopCounter = 0
    while True:
        # Create a list of lists to hold the points in each cluster
        lists = [ [] for c in clusters]
        clusterCount = len(clusters)

        # Start counting loops
        loopCounter += 1
        # For every point in the dataset ...
        for p in points:
            # Get the distance between that point and the centroid of the first
            # cluster.
            smallest_distance = getDistance(p, clusters[0].centroid)

            # Set the cluster this point belongs to
            clusterIndex = 0

            # For the remainder of the clusters ...
            for i in range(clusterCount - 1):
                # calculate the distance of that point to each other cluster's
                # centroid.
                distance = getDistance(p, clusters[i+1].centroid)
                # If it's closer to that cluster's centroid update what we
                # think the smallest distance is, and set the point to belong
                # to that cluster
                if distance < smallest_distance:
                    smallest_distance = distance
                    clusterIndex = i+1
            lists[clusterIndex].append(p)

        # Set our biggest_shift to zero for this iteration
        biggest_shift = 0.0

        # As many times as there are clusters ...
        for i in range(clusterCount):
            # Calculate how far the centroid moved in this iteration
            shift = clusters[i].update(lists[i])
            clusters[i].length = len(lists[i])
            # Keep track of the largest move from all cluster centroid updates
            biggest_shift = max(biggest_shift, shift)

        # If the centroids have stopped moving much, say we're done!
        if biggest_shift < cutoff:
            #print "Converged after %s iterations" % loopCounter
            break
    return clusters

In [27]:
def main():
    data = pd.read_csv("arcene_train.data",delimiter = ' ')
    data = np.array(data)
    #np.any(np.isnan(data))
    data = list(data)
    #print data[0]
    temp = []
    for row in data:
        temp.append([int(elements) for elements in row[:10000]])
    data = temp
    #print data[0]
    k = 50
    temp = []
    data = np.array(data)
    kpca = KernelPCA(k,kernel='poly',degree=3)
    data_x = kpca.fit_transform(data)
    data_x = list(data_x)
    for p in data_x:
        temp.append(list(p))
    data_x = temp
    #print data_x
    #print len(data_x[0])
    labels = pd.read_csv("arcene_train.labels", delimiter = ' ')
    labels = np.array(labels)
    labels = list(labels)
    temp = []
    for row in labels:
        temp.append(int(row))
    labels = temp
    points = []
    for i in range(len(data_x)):
        points.append(Point(data_x[i],labels[i]))
    num_clusters = 2
    opt_cutoff = 0.5
    clusters = kmeans(points,num_clusters,opt_cutoff)
    #print len(clusters)
    accuracy = 0.0
    count_class = dict()
    for i in range(len(clusters)):
        #print (clusters[i])
        #print clusters[i].length
        for p in clusters[i].points:
            #clusters[i] = list(clusters[i])
            if p.label in count_class.keys():
                count_class[p.label]+=1
            else:
                count_class[p.label]=1
        max_count = 0
        for key in count_class:
            if count_class[key]> max_count:
                max_count = count_class[key]
        accuracy += max_count
        count_class.clear()
    accuracy = accuracy/len(data)
    print accuracy

In [28]:
main()

0.606060606061
